## MASTER DATA SCIENCE: NUCLIO
## PROFESOR: JOSEPH GALLART
## PROJECT RENFE: APP QUE CONTENDRA LA LOGICA DEL PROGRAMA WEB, ESTA LLAMARA AL MODELO CREADO EN EL PASO 2

In [ ]:
from flask import Flask, request, jsonify, render_template
import pickle
import numpy as np
import pandas as pd
from datetime import datetime, timedelta, date
from sklearn.preprocessing import LabelEncoder,MinMaxScaler, StandardScaler


In [ ]:

app= Flask(__name__)

@app.route('/')
def home():
    return render_template('index.html')



@app.route('/predict', methods=['POST'])
def predict(): 
    try:
        
        valores=request.form.to_dict()

        # %%
        #valores= {'CIUDAD_ORIGEN': 'Madrid', 'CIUDAD_DESTINO': 'Barcelona', 'FECHA_INICIO': '2021-06-14', 'FECHA_INICIO_HORA':'9'}

        # %%
        lista_variables=pickle.load(open('modelos/variables_importantes','rb'))


        # %%
        lista_variables


        # %%
        df_tiempo_destino=pickle.load(open('datasets/df_tiempo_destino','rb'))


        # %%
        valores


        # %%
        df_input= pd.DataFrame([valores])


        # %%
        #Tenemos que agregar la hora al viaje para hacer la resta de la fecha de viaje y la hora que se inserta la fecha
        hora_viaje=int(df_input["FECHA_INICIO_HORA"][0])


        # %%
        hora_viaje


        # %%
        df_input["FECHA_INICIO"]=pd.DatetimeIndex(df_input["FECHA_INICIO"])+timedelta(hours=hora_viaje)


        # %%
        df_input


        # %%
        df_input["DIF_INI_BUS"]=(df_input["FECHA_INICIO"]- pd.Timestamp.now())/np.timedelta64(1,'m')


        # %%
        df_input


        # %%
        df_input["CIUDAD_ORIGEN"]=df_input["CIUDAD_ORIGEN"].apply(lambda x:x.upper())
        df_input


        # %%
        df_input["CIUDAD_DESTINO"]=df_input["CIUDAD_DESTINO"].apply(lambda x:x.upper())
        df_input


        # %%
        df_input["CIUDAD_ORIGEN_SEVILLA"]=0
        df_input["CIUDAD_DESTINO_SEVILLA"]=0

        if df_input["CIUDAD_ORIGEN"][0]=="SEVILLA":
            df_input["CIUDAD_ORIGEN_SEVILLA"]=1

        if df_input["CIUDAD_DESTINO"][0]=="SEVILLA":
            df_input["CIUDAD_DESTINO_SEVILLA"]=1


        # %%
        df_input


        # %%
        lista_variables


        # %%
        df_tiempo_destino


        # %%
        ### Ahora haremos un join para obtener el tiempo de viaje y el tipo de tren
        df_junto =pd.merge(df_input,df_tiempo_destino,on=["CIUDAD_ORIGEN","CIUDAD_DESTINO"])


        # %%
        df_junto


        # %%
        df_junto["DURACION_VIAJE"]=pd.to_datetime(df_junto["TIEMPO_VIAJE"],unit="m").dt.strftime("%H:%M")
        df_junto


        # %%
        def creando_tipo_tren_billete(tipo_tren, tipo_billete):
            df_output=df_junto.copy(deep=True)
            df_output["CLASE"]=tipo_tren
            df_output["TIPO_TARIFA"]=tipo_billete
            return df_output


        # %%
        df_preferente_flexible=creando_tipo_tren_billete("Preferente","Flexible")
        df_turista_flexible=creando_tipo_tren_billete("Turista","Flexible")


        # %%
        df_renfe_1=pd.concat([df_preferente_flexible,df_turista_flexible], ignore_index=True)
        df_renfe_1


        # %%
        df_renfe_2=pd.get_dummies(data=df_renfe_1,
                                        prefix=None, 
                                        prefix_sep="_",
                                        columns=["CLASE","TIPO_TARIFA"],
                                        dtype="int8")
        df_renfe_2


        # %%
        df_renfe_3 = pd.concat([df_renfe_2,df_renfe_1["CLASE"], df_renfe_1["TIPO_TARIFA"]], axis=1)
        df_renfe_3


        # %%
        df_renfe_3_otro_clase=df_renfe_3.copy()


        # %%
        df_renfe_3_otro_clase["CLASE_Preferente"]=0


        # %%
        df_renfe_3_otro_clase["CLASE_Turista"]=0


        # %%
        df_renfe_3_otro_clase["CLASE"]="Otro"


        # %%
        df_renfe_3_otro_tipo_tarifa=df_renfe_3.copy()


        # %%
        df_renfe_3_otro_tipo_tarifa["TIPO_TARIFA_Flexible"]=0


        # %%
        df_renfe_3_otro_tipo_tarifa["TIPO_TARIFA"]="Otro"


        # %%
        df_renfe_3_otro_clase


        # %%
        df_renfe_3_otro_clase.drop_duplicates(inplace=True)


        # %%
        df_renfe_3_otro_clase


        # %%
        df_renfe_3_otro_tipo_tarifa


        # %%
        df_renfe_4=pd.concat([df_renfe_3,df_renfe_3_otro_clase,df_renfe_3_otro_tipo_tarifa], ignore_index=True)


        # %%
        df_renfe_4


        # %%
        ### Falta transformar los valores numéricos que no han sido parte del OHE en una misma escala
        ### Usaremos la misma transformación hecha en nuestro pre-procesamiento, para ello importamos las transformaciones
        fichero= open('transformaciones/minMax_dif_ini_bus','rb')
        minMax_dif_ini_bus=pickle.load(fichero)
        fichero.close()

        fichero= open('transformaciones/minMax_inicio_hora','rb')
        minMax_inicio_hora=pickle.load(fichero)
        fichero.close()

        fichero= open('transformaciones/minMax_tiempo_viaje','rb')
        minMax_tiempo_viaje=pickle.load(fichero)
        fichero.close()


        # %%
        df_renfe_5=df_renfe_4.copy(deep=True)


        # %%
        df_renfe_5["DIF_INI_BUS"]=minMax_dif_ini_bus.transform(df_renfe_5["DIF_INI_BUS"].values.reshape(-1,1))

        df_renfe_5["FECHA_INICIO_HORA"]=minMax_inicio_hora.transform(df_renfe_5["FECHA_INICIO_HORA"].values.reshape(-1,1))

        df_renfe_5["TIEMPO_VIAJE"]=minMax_tiempo_viaje.transform(df_renfe_5["TIEMPO_VIAJE"].values.reshape(-1,1))


        # %%
        df_renfe_5


        # %%
        ### Ya tenemos todo! separamos este dataset en dos:
        #### 1. El que entrará al modelo para predecir
        #### 2. El que se mostrará al cliente 
        #cargamos el modelo
        df_predict=df_renfe_5[lista_variables]


        # %%
        df_predict


        # %%
        fichero= open('modelos/modelo_rf_final','rb')


        # %%
        model=pickle.load(fichero)


        # %%
        prediccion = pd.DataFrame(model.predict(df_predict), columns=['PREDICCION'], index=df_predict.index)


        # %%
        prediccion["PREDICCION"]=round(prediccion["PREDICCION"],2)


        # %%
        df_renfe_resultado=df_renfe_5.copy(deep=True)


        # %%
        df_renfe_resultado.head()


        # %%
        df_renfe_resultado=df_renfe_5[["CIUDAD_ORIGEN","CIUDAD_DESTINO","TIPO_TREN","DURACION_VIAJE","CLASE","TIPO_TARIFA"]]


        # %%
        df_renfe_resultado_final=df_renfe_resultado.join(prediccion)


        # %%
        df_renfe_resultado_final


        # %%
        df_renfe_resultado_final.sort_values(by=["CIUDAD_ORIGEN","CIUDAD_DESTINO","TIPO_TREN"],inplace=True)


        # %%
        df_renfe_resultado_final.reset_index(drop=True, inplace=True)


        # %%
        df_renfe_resultado_final


        # %%
        
        return render_template("index.html", Resultado=[df_renfe_resultado_final.to_html(classes='renfe', header="true")])
    
    except:
        return render_template("index.html", Error="RUTA NO ENCONTRADA,ORIGEN: "+valores["CIUDAD_ORIGEN"]+" DESTINO: "+valores["CIUDAD_DESTINO"] )
        

if __name__=="__main__":
    app.run()